<a href="https://colab.research.google.com/github/Sahan-Daksh/Intellihack_44Below_Task3/blob/main/FineTuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers bitsandbytes accelerate datasets peft unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 105.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# ✅ Step 1: Load Dataset
data_path = "2501.12948v1_qa_pairs-2.json"  # Update with your dataset path
dataset = load_dataset("json", data_files=data_path)

# ✅ Step 2: Split Dataset into Train & Test
dataset = dataset["train"].train_test_split(test_size=0.1)

# ✅ Step 3: Load Tokenizer
model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Step 4: Define Preprocessing Function
def preprocess_data(examples):
    """Formats instruction-based dataset into a dictionary for training."""
    prompts = []
    outputs = []

    for instruction, input_text, output in zip(examples["instruction"], examples["input"], examples["output"]):
        prompt = f"Instruction: {instruction}\nInput: {input_text}\nOutput: "

        # Tokenize input (Prompt)
        tokenized_prompt = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)

        # Tokenize output (Expected response)
        tokenized_output = tokenizer(output, padding="max_length", truncation=True, max_length=512)

        # Append to lists
        prompts.append(tokenized_prompt["input_ids"])
        outputs.append(tokenized_output["input_ids"])

    # ✅ Return a dictionary instead of a list (Fixes TypeError)
    return {
        "input_ids": prompts,
        "labels": outputs
    }

# ✅ Step 5: Apply Tokenization to the Dataset
train_dataset = dataset["train"].map(
    preprocess_data,
    batched=True,
    remove_columns=["instruction", "input", "output"]
)

test_dataset = dataset["test"].map(
    preprocess_data,
    batched=True,
    remove_columns=["instruction", "input", "output"]
)

# ✅ Step 6: Print Example to Verify
print(train_dataset[0])  # Check structure



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/93 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

{'input_ids': [16664, 25, 3555, 525, 279, 6028, 11513, 304, 4862, 264, 897, 1614, 369, 3950, 9471, 1667, 71278, 6832, 11, 323, 1246, 653, 1493, 11513, 5421, 279, 8084, 5068, 315, 279, 1614, 5267, 2505, 25, 10440, 11, 279, 897, 1614, 198, 19798, 398, 33353, 279, 4271, 315, 9471, 2474, 432, 27193, 1817, 3019, 315, 279, 2711, 1882, 13, 16151, 264, 6915, 24321, 2627, 897, 1614, 374, 48088, 5000, 11, 892, 3643, 432, 17003, 369, 279, 198, 2528, 311, 5367, 7887, 7269, 13, 5976, 24708, 10850, 748, 6200, 2393, 39913, 389, 4862, 264, 897, 198, 5097, 25, 220, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 15164

In [ ]:
import torch
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

import os
os.environ["WANDB_DISABLED"] = "true"


# ✅ Load Qwen Model with 4-bit Quantization (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# ✅ Prepare Model for Training
model = prepare_model_for_kbit_training(model)

# ✅ Attach LoRA Adapters
lora_config = LoraConfig(
    r=8,  # LoRA rank
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# ✅ Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=1,  # Small batch size due to limited VRAM
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,  # Simulate larger batch size
    logging_dir="./logs",
    fp16=True,
    num_train_epochs=3,
    save_total_limit=1,
    optim="adamw_bnb_8bit",
    remove_unused_columns=False,  # Prevent Trainer from removing dataset columns
)

# ✅ Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# ✅ Free GPU Memory Before Training
torch.cuda.empty_cache()

# ✅ Start Training
trainer.train()

# ✅ Save LoRA Adapters
model.save_pretrained("./fine_tuned_qwen_lora")
tokenizer.save_pretrained("./fine_tuned_qwen_lora")

print("🎉 Fine-tuning completed with QLoRA! LoRA adapters saved.")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,9.409946
2,No log,7.291879


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


🎉 Fine-tuning completed with QLoRA! LoRA adapters saved.


In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

# Load base model
model_name = "Qwen/Qwen2.5-3B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cpu")

# Load fine-tuned LoRA adapters
fine_tuned_lora_path = "./fine_tuned_qwen_lora"
model = PeftModel.from_pretrained(base_model, fine_tuned_lora_path)

# Merge LoRA weights into the base model
model = model.merge_and_unload()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import torch
import os
from transformers import AutoModelForCausalLM
from peft import PeftModel

# ✅ Check RAM & Disk Space Before Merging
os.system("free -h")  # Show available RAM
os.system("df -h")    # Show disk space usage

# ✅ Load Base Model with Memory Optimization
model_name = "Qwen/Qwen2.5-3B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Reduce memory usage
    device_map="cpu"  # Load on CPU to prevent GPU crashes
)

# ✅ Free GPU Memory Before Merging (if using Colab/Kaggle)
torch.cuda.empty_cache()

# ✅ Load Fine-Tuned LoRA Adapters
fine_tuned_lora_path = "./fine_tuned_qwen_lora"
model = PeftModel.from_pretrained(base_model, fine_tuned_lora_path).to("cpu")

# ✅ Merge LoRA Weights into Base Model
print("Merging LoRA weights into base model... This may take some time.")
model = model.merge_and_unload()

# ✅ Save the Fully Merged Model
merged_model_path = "./qwen2.5_3b_merged"
model.save_pretrained(merged_model_path)

print(f"🎉 LoRA weights merged successfully! Model saved at {merged_model_path}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Merging LoRA weights into base model... This may take some time.
🎉 LoRA weights merged successfully! Model saved at ./qwen2.5_3b_merged


In [ ]:
!pip install llama-cpp-python transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4552826 sha256=208e3fd53ee10fc02ea8f81ac8c3e4efb31a87d0ed39f4ef4e8420456546334d
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama-cpp-python


In [ ]:
# Install required packages
!apt-get install -y cmake build-essential
!pip install llama-cpp-python
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!make
%cd ..


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
fatal: destination path 'llama.cpp' already exists and is not an empty directory.
/content/llama.cpp
Makefile:2: *** The Makefile build is deprecated. Use the CMake build instead. For more details, see https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md.  Stop.
/content


In [ ]:
# Define paths
merged_model_path = "./qwen2.5_3b_merged"
output_gguf_path = "./qwen2.5_3b_finetuned.gguf"

# Run llama.cpp conversion script
!python llama.cpp/convert.py --model_dir {merged_model_path} --output_dir {output_gguf_path}

print(f"🎉 GGUF model saved at: {output_gguf_path}")


python3: can't open file '/content/llama.cpp/convert.py': [Errno 2] No such file or directory
🎉 GGUF model saved at: ./qwen2.5_3b_finetuned.gguf


In [ ]:
!ls -lh ./qwen2.5_3b_finetuned.gguf


ls: cannot access './qwen2.5_3b_finetuned.gguf': No such file or directory
